In [1]:
import os
import tarfile
import urllib.request


MODEL_URL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz"
MODEL_DIR = "models"
MODEL_FILE = os.path.join(MODEL_DIR, "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz")
MODEL_NAME = "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8"

if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

urllib.request.urlretrieve(MODEL_URL, MODEL_FILE)

with tarfile.open(MODEL_FILE, "r:gz") as tar:
    tar.extractall(path=MODEL_DIR)


In [2]:
import os

IMAGE_DIR = "images"
if not os.path.exists(IMAGE_DIR):
    os.makedirs(IMAGE_DIR)



In [4]:
import cv2
import numpy as np
import tensorflow as tf

In [5]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import matplotlib.pyplot as plt
#do ustawienia
LABEL_MAP_PATH = "/home/maershal/.keras/datasets/mscoco_label_map.pbtxt"
import matplotlib
matplotlib.use('TkAgg')
#mozna innego backednu, ten mi dzialal bo jakis blad z matplotlibem mialem
LABEL_MAP = label_map_util.load_labelmap(LABEL_MAP_PATH)
CATEGORIES = label_map_util.convert_label_map_to_categories(LABEL_MAP, max_num_classes=90, use_display_name=True)
CATEGORY_INDEX = label_map_util.create_category_index(CATEGORIES)

# Wczytanie modelu
model = tf.saved_model.load(os.path.join(MODEL_DIR, MODEL_NAME, "saved_model"))

# Przetwarzanie zdjęć
for image_file in os.listdir(IMAGE_DIR):
    image_path = os.path.join(IMAGE_DIR, image_file)
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    input_tensor = tf.convert_to_tensor(image_rgb)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = model(input_tensor)

    # Usunięcie zbędnych wymiarów
    num_detections = int(detections.pop("num_detections"))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections["num_detections"] = num_detections

    # Konwersja wykrytych klas na typ int
    detections["detection_classes"] = detections["detection_classes"].astype(np.int64)

    # Wizualizacja wyników detekcji
    image_with_detections = image_rgb.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_with_detections,
        detections["detection_boxes"],
        detections["detection_classes"],
        detections["detection_scores"],
        CATEGORY_INDEX,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.30,
        agnostic_mode=False)

    # Wyświetlenie wyników
    image_with_detections = cv2.cvtColor(image_with_detections, cv2.COLOR_RGB2BGR)

    # ...
    # Wyświetlenie wyników
    plt.figure(figsize=(12, 8))
    plt.imshow(image_with_detections)
    plt.title(f"Detekcja obiektów: {image_file}")
    plt.show()




ModuleNotFoundError: No module named 'object_detection'